In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
from tqdm import tqdm
from gensim.models import Word2Vec

In [ ]:
# Mecab 실행 환경 구축 
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
!pip install konlpy   # konlpy 라이브러리 설치 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model_name = 'word2vec_wellness.model'
model = Word2Vec.load('/content/drive/MyDrive/LAMDA/chatbot/dataset/' + model_name)

In [ ]:
word_vectors = model.wv
# word_vectors

In [ ]:
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]    # 단어 리스트 생성 
len(word_vectors_list), len(word_vectors_list[0])

(20932, 100)

In [ ]:
model.wv.most_similar('예민해')

[('가도', 0.9835881590843201),
 ('자살을', 0.9831641912460327),
 ('없는데도', 0.9830091595649719),
 ('속상해서', 0.9827791452407837),
 ('시어머니', 0.9827532768249512),
 ('체력이', 0.9827108383178711),
 ('처럼', 0.9825912714004517),
 ('예민하게', 0.9825780987739563),
 ('사람들이랑', 0.9825144410133362),
 ('모습이', 0.9823926687240601)]

#### Keyword-Intent 매핑 테이블 정의 

In [3]:
tmp = pd.read_csv('/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent_keyword.csv')
data = tmp[['intent', 'intent_keyword']]
data

,intent,intent_keyword
0,우울감,우울감/우울
1,우울감,우울감/우울
2,우울감,우울감/우울
3,우울감,우울감/우울
4,우울감,우울감/우울
...,...,...
19671,불안,불안/압박감
19672,불안,불안/압박감
19673,불안,불안/압박감
19674,불안,불안/압박감


In [5]:
ik_list = list(data.intent_keyword.unique())
intent_list = list(data.intent.unique())

In [6]:
ik_list[0].split('/')[0] != ik_list[1].split('/')[0]

False

In [7]:
intent_list

['우울감',
 '슬픔',
 '외로움',
 '분노',
 '무기력',
 '감정조절이상',
 '상실감',
 '식욕저하',
 '식욕증가',
 '불면',
 '초조함',
 '피로',
 '죄책감',
 '집중력저하',
 '자신감저하',
 '자존감저하',
 '절망감',
 '자살충동',
 '불안']

In [ ]:
len(ik_list)

205

In [ ]:
map_tb = []
start = 0; end = len(ik_list)

for idx in range(len(intent_list)):
  map_tb.append([])

  for idx2 in range(start, end):
    if ik_list[idx2].split('/')[0] != intent_list[idx]:
      start = idx2 
      break 
    
    map_tb[idx].append(ik_list[idx2].split('/')[1])

len(map_tb)

19

In [ ]:
# 검증
count = 0 
for idx in range(len(map_tb)):
  count += len(map_tb[idx])

count

205

In [ ]:
map_tb[0] = ['우울', '멍해', '무기력', '불면', '재미없어', '식욕저하', '자존감', '패배감', '집중력']
map_tb[1] = ['슬퍼', '속상해', '눈물', '죄책감', '그리워', '서러워', '비참해', '안타까워', '허망해', '허무해', '절망', '억울해', '후회돼', '섭섭해']
map_tb[2] = ['외로워', '쓸쓸해', '무의미해', '적적해', '울적해', '고단해', '외톨이', '고독해', '겉돌아']
map_tb[3] = ['화나', '원망스러워', '불만', '미워', '불쌍해', '증오', '허망해']
map_tb[4] = ['재미없어', '무기력', '괴로워', '귀찮아', '허무해', '우울해']
map_tb[5] = ['막말']
map_tb[6] = ['공허해' ,'허무해', '허탈해', '자괴감', '패배감', '외로움', '고독해', '무기력']
map_tb[7] = ['식욕저하', '무기력', '예민해', '메슥거려', '소화불량']
map_tb[8] = ['폭식', '과체중']
map_tb[9] = ['불면', '만성피로', '몽롱해', '늘어져', '비몽사몽이야']
map_tb[10] = ['안절부절', '예민해', '짜증', '혼란스러워', '느려', '무거워', '떨려', '재미없어', '답답해', '신경질', '감정기복', '걱정', '욱해']
map_tb[11] = ['나른해', '슬럼프', '무거워', '처져', '누워있어', '멍해' , '졸려', '찌뿌둥해']
map_tb[12] = ['죄책감', '죄인', '잘못', '억울해', '불쌍해', '자괴감', '원망스러워', '후회', '허탈해', '질책', '자책해', '배신감', '패배감']
map_tb[13] = ['책', '티비', '재미없어', '피곤해', '불안정해']
map_tb[14] = ['느긋해', '걱정돼', '대충', '소심']
map_tb[15] = ['하찮아', '패배자', '냉소적이야', '좌절', '감시', '해코지', '손가락질', '째려봐', '괴로워', '피해의식', '한심해']
map_tb[16] = ['좌절', '절망스러워', '고통', '실망', '비참', '나약해', '하찮아']
map_tb[17] = ['하찮아']
map_tb[18] = ['불안', '긴장', '낭떠러지', '두려워', '울렁거려', '압박감']

#### 단어 행렬

In [ ]:
from tqdm import tqdm
from konlpy.tag import Mecab 

mecab = Mecab() 

In [ ]:
index = list(vocabs)

word2vec_df = pd.DataFrame(word_vectors_list, index=index)
word2vec_df.to_csv('/content/drive/MyDrive/LAMDA/chatbot/dataset/word_vectors.csv', index=index)

In [ ]:
word2vec_df2 = pd.read_csv('/content/drive/MyDrive/LAMDA/chatbot/dataset/word_vectors.csv', index_col=0)
word2vec_df2 

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
임신해서,-0.046904,-0.026849,0.049267,0.010699,-0.025946,0.023811,0.044171,-0.024407,0.006207,0.050334,...,0.007118,0.011405,0.006556,0.009071,0.018188,-0.006998,0.002074,0.002532,0.004965,0.007069
우울해,-0.468959,-0.247410,0.523740,0.076209,-0.281233,0.263763,0.402840,-0.229637,0.066605,0.536349,...,0.122170,0.109942,0.107067,0.078389,0.151570,-0.051144,0.080963,0.043915,0.043112,0.086259
아이,-0.137740,-0.079064,0.163112,0.023787,-0.088077,0.073877,0.125786,-0.069803,0.020218,0.155874,...,0.033755,0.038485,0.038217,0.019890,0.044278,-0.012611,0.025049,0.011334,0.012888,0.022187
가지고,-0.210793,-0.108406,0.226248,0.033736,-0.127683,0.112832,0.170938,-0.099354,0.035048,0.237834,...,0.053398,0.046888,0.044690,0.029181,0.068883,-0.022268,0.029727,0.014077,0.015745,0.039402
나서,-1.046773,-0.560318,1.158926,0.144812,-0.638555,0.568429,0.882015,-0.530070,0.176712,1.186034,...,0.275426,0.263903,0.234547,0.160811,0.327434,-0.102541,0.173318,0.094303,0.089159,0.171616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
애원했어요,-0.009999,-0.008882,0.013293,-0.001129,-0.004982,0.011120,0.013933,-0.002735,-0.001183,0.013870,...,0.005493,0.007292,-0.001459,-0.001855,0.004568,-0.000752,-0.001073,0.002148,0.002648,0.004386
집인데,-0.013883,-0.006564,0.017393,0.006288,-0.009999,0.008316,0.015410,-0.009546,0.000377,0.010740,...,0.008287,-0.000240,0.005341,0.006782,0.003138,-0.004395,0.003100,0.002633,0.000402,-0.002529
나쁜일이,-0.015550,-0.009928,0.012429,0.003352,-0.012044,0.007836,0.015650,-0.002630,0.006147,0.012550,...,0.007616,0.005950,0.000467,0.000150,0.009511,0.000140,0.001788,-0.002828,-0.003131,0.005665
잠못자고,-0.014985,-0.003637,0.015408,0.003619,-0.004640,0.006874,0.013342,-0.008550,0.004644,0.013154,...,0.001468,0.005369,0.002782,-0.001668,0.004174,-0.006041,-0.001419,-0.002606,0.002065,0.003679


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(word2vec_df2, word2vec_df2)
len(cosine_sim), len(cosine_sim)

(20932, 20932)

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=list(vocabs), columns=list(vocabs))
cosine_sim_df

,임신해서,우울해,아이,가지고,나서,가졌는데,기분,하나도,안,좋고,...,부탁했어,무서워져서,당시처럼,두려워서,살려달라고,애원했어요,집인데,나쁜일이,잠못자고,불안한건
임신해서,1.000000,0.992804,0.991668,0.992446,0.992400,0.965582,0.992720,0.992333,0.990686,0.991972,...,0.840454,0.825340,0.825859,0.816572,0.954931,0.892579,0.913191,0.934662,0.927960,0.914166
우울해,0.992804,1.000000,0.998739,0.999437,0.999741,0.975604,0.999743,0.999744,0.998852,0.999242,...,0.843113,0.835045,0.839352,0.816914,0.963826,0.899339,0.914070,0.937715,0.934015,0.921678
아이,0.991668,0.998739,1.000000,0.998554,0.998977,0.975844,0.998658,0.998667,0.998323,0.998206,...,0.838633,0.836469,0.833555,0.816491,0.961622,0.895674,0.917680,0.940628,0.933444,0.920044
가지고,0.992446,0.999437,0.998554,1.000000,0.999557,0.975664,0.999478,0.999349,0.998786,0.998984,...,0.845978,0.835881,0.835557,0.812394,0.964157,0.896731,0.915355,0.940050,0.933845,0.921819
나서,0.992400,0.999741,0.998977,0.999557,1.000000,0.975832,0.999769,0.999753,0.999278,0.999333,...,0.844776,0.836661,0.837961,0.817805,0.963702,0.897678,0.915519,0.938749,0.933509,0.922193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
애원했어요,0.892579,0.899339,0.895674,0.896731,0.897678,0.889154,0.897647,0.897589,0.899344,0.896697,...,0.801089,0.769417,0.722715,0.737603,0.876319,1.000000,0.807924,0.831730,0.838056,0.803538
집인데,0.913191,0.914070,0.917680,0.915355,0.915519,0.892940,0.914749,0.914158,0.913703,0.914510,...,0.768355,0.793104,0.714849,0.718002,0.876561,0.807924,1.000000,0.851756,0.857135,0.848951
나쁜일이,0.934662,0.937715,0.940628,0.940050,0.938749,0.911579,0.938715,0.938379,0.940135,0.939111,...,0.803932,0.796388,0.798752,0.750742,0.900814,0.831730,0.851756,1.000000,0.873388,0.872589
잠못자고,0.927960,0.934015,0.933444,0.933845,0.933509,0.904802,0.933758,0.934030,0.931926,0.934245,...,0.802307,0.780765,0.790140,0.771495,0.885008,0.838056,0.857135,0.873388,1.000000,0.877228


In [ ]:
cosine_sim_df.to_csv('/content/drive/MyDrive/LAMDA/chatbot/dataset/cosine_sim.csv', index=index, header=1)

In [ ]:
keyword_list = []

for keywords in map_tb:
  for keyword in keywords: 
    keyword_list.append(keyword)

In [ ]:
idx_list = []   # keyword 카테고리에 해당하는 인덱스 저장 

for cate in keyword_list:
  idx = 0
  if cate not in vocabs: 
    print(cate)
    continue
   
  for v in vocabs:
    if v == cate:
      idx_list.append(idx)
    idx += 1 

len(idx_list), len(keyword_list)

(134, 134)

In [ ]:
keyword_cate_sim = cosine_sim_df.iloc[idx_list]
keyword_cate_sim

,임신해서,우울해,아이,가지고,나서,가졌는데,기분,하나도,안,좋고,...,부탁했어,무서워져서,당시처럼,두려워서,살려달라고,애원했어요,집인데,나쁜일이,잠못자고,불안한건
우울,0.984836,0.991904,0.990896,0.991761,0.991835,0.970671,0.991518,0.992093,0.990366,0.990999,...,0.840872,0.838478,0.822667,0.812431,0.960144,0.897620,0.910627,0.928500,0.926295,0.914483
멍해,0.991312,0.998862,0.998203,0.998962,0.999201,0.975388,0.998930,0.998960,0.998841,0.998589,...,0.845050,0.834007,0.835870,0.815437,0.961445,0.899950,0.913152,0.937526,0.934910,0.922792
무기력,0.848527,0.853238,0.848217,0.853535,0.851997,0.837637,0.851980,0.850342,0.848070,0.850184,...,0.710892,0.723802,0.673155,0.651438,0.836077,0.748508,0.822011,0.792031,0.778817,0.791528
불면,0.985333,0.992254,0.991989,0.992001,0.992367,0.971801,0.992346,0.992461,0.991890,0.990641,...,0.823314,0.840176,0.827783,0.819011,0.953705,0.887352,0.910999,0.924572,0.929265,0.929576
재미없어,0.989576,0.995910,0.994887,0.995452,0.995705,0.970108,0.995999,0.995668,0.994468,0.994930,...,0.835326,0.827958,0.837594,0.809622,0.963049,0.894931,0.914166,0.937814,0.927176,0.918984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
긴장,0.991188,0.999060,0.998369,0.998931,0.999305,0.974538,0.999060,0.999085,0.999070,0.998982,...,0.844815,0.839810,0.837562,0.816083,0.962961,0.899273,0.914062,0.937101,0.934621,0.922416
낭떠러지,0.951494,0.963564,0.962085,0.964787,0.964269,0.953598,0.963795,0.964540,0.963597,0.962585,...,0.816091,0.820775,0.838431,0.771751,0.928626,0.862597,0.874946,0.911624,0.909660,0.906690
두려워,0.992153,0.998049,0.996516,0.997522,0.997994,0.971459,0.998136,0.997903,0.996766,0.997250,...,0.843131,0.837216,0.829118,0.816864,0.960911,0.900955,0.915311,0.935566,0.933055,0.922400
울렁거려,0.991192,0.997705,0.997175,0.997632,0.997904,0.972566,0.997767,0.997687,0.996827,0.997409,...,0.847771,0.835106,0.830602,0.814090,0.963518,0.897667,0.914090,0.940148,0.932755,0.921079


In [ ]:
test_dataset = pd.read_csv('/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent_keyword.csv')
test_dataset

,intent,keyword(임상키워드),utterance(2차),intent_label,intent_keyword,ik_label
0,우울감,우울,임신해서 우울해,0,우울감/우울,0
1,우울감,우울,아이 가지고 나서 우울해,0,우울감/우울,0
2,우울감,우울,아이 가졌는데 기분 하나도 안 좋고 울적해,0,우울감/우울,0
3,우울감,우울,임신했는데 남편이 하나도 안 챙겨줘서 우울하다,0,우울감/우울,0
4,우울감,우울,진단 결과 안 좋게 나올 것 같아서 우울해,0,우울감/우울,0
...,...,...,...,...,...,...
19671,불안,압박감,그래도 잠못자고 불안한건 여전해요.,18,불안/압박감,204
19672,불안,압박감,불안함에 항상 시달리니까 잠도 못잤어요.,18,불안/압박감,204
19673,불안,압박감,불안하고 초조해서 잠이 안 와.,18,불안/압박감,204
19674,불안,압박감,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,18,불안/압박감,204
